# Singapore maritime dataset frames generation
a

a

a

a


In [1]:
from scipy.io import loadmat
from os import listdir
from os.path import isfile, join
import cv2
from PIL import Image, ImageDraw
import PIL
import math

# dimensions to save the images
x_size = 400
y_size = 225

Load the videos paths for both onboard and onshore data and generate dictionaries with it.

In [2]:
# Set the paths for the video files and ground truth files

"""
VIDEOS_PATH_ONSHORE = "../../Dataset/VIS_Onshore/Videos"
HORIZON_ANNOTATIONS_ONSHORE_PATH = "../../Dataset/VIS_Onshore/HorizonGT"
VIDEO_FRAMES_PATH_ONSHORE = '../../Dataset/VIS_Onshore_frames/'
"""

VIDEOS_PATH_ONBOARD = "../../Dataset/VIS_Onboard/Videos"
HORIZON_ANNOTATIONS_ONBOARD_PATH = "../../Dataset/VIS_Onboard/HorizonGT"
VIDEO_FRAMES_PATH_ONBOARD = '../../Dataset/VIS_Onboard/VIS_Onboard_frames/'

In [3]:
# video_files_onshore = [join(VIDEOS_PATH_ONSHORE, f) for f in listdir(VIDEOS_PATH_ONSHORE) 
#                        if isfile(join(VIDEOS_PATH_ONSHORE, f))]

video_files_onboard = [join(VIDEOS_PATH_ONBOARD, f) for f in listdir(VIDEOS_PATH_ONBOARD) 
                       if isfile(join(VIDEOS_PATH_ONBOARD, f))]

Create dictionaries for each video in the form video_name:video_path

In [4]:
"""
video_files_onshore_dict = {}
for f in listdir(VIDEOS_PATH_ONSHORE):
    if isfile(join(VIDEOS_PATH_ONSHORE, f)):
        video_files_onshore_dict[f.split('.')[0]] = join(VIDEOS_PATH_ONSHORE, f)
"""        
video_files_onboard_dict = {}
for f in listdir(VIDEOS_PATH_ONBOARD):
    if isfile(join(VIDEOS_PATH_ONBOARD, f)):
        video_files_onboard_dict[f.split('.')[0]] = join(VIDEOS_PATH_ONBOARD, f)

Load the ground truth files paths for both onboard and onshore data and generate dictionaries with it.

In [5]:
"""
horizon_gt_files_onshore_dict = {}
for f in listdir(HORIZON_ANNOTATIONS_ONSHORE_PATH):
    if isfile(join(HORIZON_ANNOTATIONS_ONSHORE_PATH, f)):
        horizon_gt_files_onshore_dict[f.split('.')[0].replace('_HorizonGT','')] = join(HORIZON_ANNOTATIONS_ONSHORE_PATH, f)
"""        
horizon_gt_files_onboard_dict = {}
for f in listdir(HORIZON_ANNOTATIONS_ONBOARD_PATH):
    if isfile(join(HORIZON_ANNOTATIONS_ONBOARD_PATH, f)):
        horizon_gt_files_onboard_dict[f.split('.')[0].replace('_HorizonGT','')] = join(HORIZON_ANNOTATIONS_ONBOARD_PATH, f)

---------------------------

## Do some checks

#### Numbers of videos and ground truth files

Do some sanity checks to see if there are equal numbers of videos and ground truth files.

In [6]:
# print('Number of onshore videos: ', len(video_files_onshore_dict))
# print('Number of onshore ground truth files: ', len(horizon_gt_files_onshore_dict))

print('Number of onboard videos: ', len(video_files_onboard_dict))
print('Number of onboard ground truth files: ', len(horizon_gt_files_onboard_dict))

Number of onboard videos:  11
Number of onboard ground truth files:  11


So there are videos without ground truth files and ground truth files without videos. These unlabelled data might be good for testing later. Let's find these videos and ground truth files.

In [7]:
# ground truth files are missing - find the corresponding videos
# videos are missing - find the corresponding ground truth files
"""
missing_files_onshore = []
for key in video_files_onshore_dict.keys():
    if key not in horizon_gt_files_onshore_dict:
        missing_files_onshore.append(key)
        
for key in horizon_gt_files_onshore_dict.keys():
    if key not in video_files_onshore_dict:
        missing_files_onshore.append(key)
        
print("Unlabelled onshore videos: ", missing_files_onshore)
"""

missing_files_onboard = []
for key in video_files_onboard_dict.keys():
    if key not in horizon_gt_files_onboard_dict:
        missing_files_onboard.append(key)
        
for key in horizon_gt_files_onboard_dict.keys():
    if key not in video_files_onboard_dict:
        missing_files_onboard.append(key)

print("Unlabelled onboard videos: ", missing_files_onboard)


# set whether to remove or not the missing videos from the frames generation later
remove_missing_files = True
if remove_missing_files:
    #for key in missing_files_onshore:
    #    del video_files_onshore_dict[key]
    #    del horizon_gt_files_onshore_dict[key]
    for key in missing_files_onboard:
        del video_files_onboard_dict[key]
        del horizon_gt_files_onboard_dict[key]

    print()
    print('Size of video dictionaries after removing the videos without ground truth:')
    """
    print('Number of onshore videos: ', len(video_files_onshore_dict))
    print('Number of onshore ground truth files: ', len(horizon_gt_files_onshore_dict))
    """

    print('Number of onboard videos: ', len(video_files_onboard_dict))
    print('Number of onboard ground truth files: ', len(horizon_gt_files_onboard_dict))

Unlabelled onboard videos:  []

Size of video dictionaries after removing the videos without ground truth:
Number of onboard videos:  11
Number of onboard ground truth files:  11



#### Count video frame number and GT data frame number

Do some sanity checks to see if there are equal numbers of videos and ground truth files.

In [8]:
# count video frames
for video_key in video_files_onboard_dict:
    vidcap = cv2.VideoCapture(video_files_onboard_dict.get(video_key))

    # get total frames of video
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("Total frames " + video_key + " : " + str(total_frames))

Total frames MVI_0788_VIS_OB : 299
Total frames MVI_0789_VIS_OB : 279
Total frames MVI_0790_VIS_OB : 600
Total frames MVI_0792_VIS_OB : 299
Total frames MVI_0794_VIS_OB : 299
Total frames MVI_0795_VIS_OB : 255
Total frames MVI_0796_VIS_OB : 299
Total frames MVI_0797_VIS_OB : 600
Total frames MVI_0799_VIS_OB : 600
Total frames MVI_0801_VIS_OB : 600
Total frames MVI_0804_VIS_OB : 299


In [62]:
# count GT data frames
for horizon_key in horizon_gt_files_onboard_dict:
    # read GT data
    data = loadmat(horizon_gt_files_onboard_dict.get(horizon_key))
    
    # get total frames of GT data
    total_frames = len(data['structXML'][0])
    print("Total GT frames " + horizon_key + " : " + str(total_frames))

Total GT frames MVI_0788_VIS_OB : 299
Total GT frames MVI_0789_VIS_OB : 279
Total GT frames MVI_0790_VIS_OB : 299
Total GT frames MVI_0792_VIS_OB : 299
Total GT frames MVI_0794_VIS_OB : 299
Total GT frames MVI_0795_VIS_OB : 299
Total GT frames MVI_0796_VIS_OB : 299
Total GT frames MVI_0797_VIS_OB : 299
Total GT frames MVI_0799_VIS_OB : 299
Total GT frames MVI_0801_VIS_OB : 299
Total GT frames MVI_0804_VIS_OB : 299


In [9]:

keys = list(horizon_gt_files_onboard_dict.keys())

for key in keys:
    #print(video_files_onboard_dict.get(video_keys[i]) + " : " + horizon_gt_files_onboard_dict.get(horizon_keys[i]))
    
    # read GT data
    data = loadmat(horizon_gt_files_onboard_dict.get(key))
    # get total frames of GT data
    total_frames_gt = len(data['structXML'][0])
    
    vidcap = cv2.VideoCapture(video_files_onboard_dict.get(key))
    # get total frames of video
    total_frames_video = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    #print("* " + str(total_frames_gt) + " - " + str(total_frames_video))
    if total_frames_gt != total_frames_video:
        del video_files_onboard_dict[key]
        del horizon_gt_files_onboard_dict[key]

print()
print('Size of video dictionaries after removing the videos with different numbers of frames and ground truth:')

print('Number of onboard videos: ', len(video_files_onboard_dict))
print('Number of onboard ground truth files: ', len(horizon_gt_files_onboard_dict))


Size of video dictionaries after removing the videos with different numbers of frames and ground truth:
Number of onboard videos:  6
Number of onboard ground truth files:  6


In [10]:
print(video_files_onboard_dict.keys())

dict_keys(['MVI_0788_VIS_OB', 'MVI_0789_VIS_OB', 'MVI_0792_VIS_OB', 'MVI_0794_VIS_OB', 'MVI_0796_VIS_OB', 'MVI_0804_VIS_OB'])


---------------------------

## Convert ALL frames of the videos into jpg images
 This is code to convert each video frame into a jpg image.

#### Example
 This cell is for converting only one video.

In [ ]:
# convert a sample onshore video

video_name = 'MVI_1478_VIS'
vidcap = cv2.VideoCapture(video_files_onshore_dict.get(video_name))
success,image = vidcap.read()
count = 0
success = True
while success:
    resized_image = cv2.resize(image, (x_size, y_size))
    cv2.imwrite(VIDEO_FRAMES_PATH_ONSHORE + video_name + "_frame%d.jpg" % count, resized_image)     # save frame as JPEG file
    success,image = vidcap.read()
    #print('Read a new frame: ', success)
    count += 1
print("Derived %d frames" % count)

In [ ]:
# convert a sample onboard video

video_name = 'MVI_0788_VIS_OB'
vidcap = cv2.VideoCapture(video_files_onboard_dict.get(video_name))
success,image = vidcap.read()
count = 0
success = True
while success:
    resized_image = cv2.resize(image, (x_size, y_size))
    cv2.imwrite(VIDEO_FRAMES_PATH_ONBOARD + video_name + "_frame%d.jpg" % count, resized_image)     # save frame as JPEG file
    success,image = vidcap.read()
    #print('Read a new frame: ', success)
    count += 1
print("Derived %d frames" % count)

#### Convert ALL frames

This cell is for converting all the videos in a folder into jpg images.

In [ ]:
# convert ALL on shore videos into images with 1 image per frame
for video_key in video_files_onshore_dict:
    #video_name = 'MVI_1478_VIS'
    vidcap = cv2.VideoCapture(video_files_onshore_dict.get(video_key))
    success,image = vidcap.read()
    count = 0
    success = True
    while success:
        resized_image = cv2.resize(image, (x_size, y_size))
        cv2.imwrite(VIDEO_FRAMES_PATH_ONSHORE + video_key + "_frame%d.jpg" % count, resized_image)     # save frame as JPEG file
        success,image = vidcap.read()
        #print('Read a new frame: ', success)
        count += 1
    print("Derived %d frames" % count)

In [ ]:
# convert ALL on board videos into images with 1 image per frame
for video_key in video_files_onboard_dict:
    #video_name = 'MVI_1478_VIS'
    vidcap = cv2.VideoCapture(video_files_onboard_dict.get(video_key))
    success,image = vidcap.read()
    count = 0
    success = True
    while success:
        resized_image = cv2.resize(image, (x_size, y_size))
        cv2.imwrite(VIDEO_FRAMES_PATH_ONBOARD + video_key + "_frame%d.jpg" % count, resized_image)     # save frame as JPEG file
        success,image = vidcap.read()
        #print('Read a new frame: ', success)
        count += 1
    print("Derived %d frames" % count)

---------------------------
## Convert every N frame of a video into jpg image

In [ ]:
# convert every N onshore videos into images with 1 image per frame
for video_key in video_files_onshore_dict:
    frame_space = 20
    vidcap = cv2.VideoCapture(video_files_onshore_dict.get(video_key))
    success,image = vidcap.read()
    count = 0
    frame_count = 0
    success = True
    while success:
        if (count == 0) or ((count+1) % frame_space == 0):
            resized_image = cv2.resize(image, (x_size, y_size))
            cv2.imwrite(VIDEO_FRAMES_PATH_ONSHORE + video_key + "_frame%d.jpg" % count, resized_image)     # save frame as JPEG file
            frame_count += 1
        success,image = vidcap.read()
        #print('Read a new frame: ', success)
        count += 1
    print(video_key)
    print("Total %d frames" % count)
    print("Derived %d frames" % frame_count)

In [22]:
# convert every N onboard videos into images with 1 image per frame
for video_key in video_files_onboard_dict:
    frame_space = 20
    vidcap = cv2.VideoCapture(video_files_onboard_dict.get(video_key))
    success,image = vidcap.read()
    count = 0
    frame_count = 0
    success = True
    while success:
        if (count == 0) or ((count+1) % frame_space == 0):
            resized_image = cv2.resize(image, (x_size, y_size))
            cv2.imwrite(VIDEO_FRAMES_PATH_ONBOARD + video_key + "R_frame%d.jpg" % count, resized_image)     # save frame as JPEG file
            frame_count += 1
        success,image = vidcap.read()
        #print('Read a new frame: ', success)
        count += 1
    print(video_key)
    print("Total %d frames" % count)
    print("Derived %d frames" % frame_count)

MVI_0788_VIS_OB
Total 299 frames
Derived 15 frames


KeyboardInterrupt: 

---------------------------

## Generate ALL GT images
 This is code to generate the GT images using all data from valid GT files.

#### Example
 This cell is for generating only one GT image.

In [27]:
# Read GT data
data = loadmat("../../Dataset/VIS_Onboard/HorizonGT/MVI_0788_VIS_OB_HorizonGT")

# Read de base image
frame_number = 12
#base = np.array(Image.open("../../Dataset/VIS_Onboard/VIS_Onboard_frames/MVI_0788_VIS_OB_frame" + str(frame_number) + ".jpg"), dtype=np.uint8)

# Get image dimension
len_x, len_y = 1920, 1080

# Create GT image
# use the line below to see the horizon line in binary image ----
# PIL.Image.new(binary chanel, (x dimension, y dimension))
gt = PIL.Image.new('1', (len_x, len_y))
# use the line below to see the horizon line in the sea image ----
# Image.open(image_path)
#gt = Image.open("../../Dataset/VIS_Onboard/VIS_Onboard_frames/MVI_0788_VIS_OB_frame" + str(frame_number) + ".jpg")

# Create a draw with the image
draw = ImageDraw.Draw(gt)

# horizon = data frame <frame_number> -> (x, y, cos alpha, sen alpha) - See Explanation of GT files
horizon = data['structXML'][0][frame_number]
print(horizon)


# ------- Create the horizon line -------
# cosine and sine from GT file horizon line
c, s = horizon[2], horizon[3]
# horizon line angle
rad = math.asin(horizon[3]) - math.radians(90)
# cosine and sine to plot horizon line
c, s = math.cos(rad), math.sin(rad)

# central point
cx = float(horizon[0][0])
cy = float(horizon[1][0])
print(cx)
print(cy)

# start point and end point
x1 = 0 # start point
y1 = cy
x2 = len_x # end point
y2 = cy

# rotated points
xr1 = c*(x1-cx) - s*(y1-cy) + cx
yr1 = s*(x1-cx) + c*(y1-cy) + cy
xr2 = c*(x2-cx) - s*(y2-cy) + cx
yr2 = s*(x2-cx) + c*(y2-cy) + cy
# ---------------------------------------

# Draw the horizon line
# draw.line((x start point, y start point, x end point, y end point), white color, 1 pixel of width)
draw.line((xr1, yr1, xr2, yr2), fill=1, width=6)

# Show the image
#gt.show()

gt = gt.resize((x_size, y_size))

# Save the image
gt.save("GTs/MVI_0788_VIS_OB_gt" + str(frame_number) + ".jpeg", "JPEG")

Dimensões do frame: 1920 x 1080
(array([[960.5]]), array([[541.01841397]]), array([[0.03016957]]), array([[0.99954479]]))
960.5
541.0184139689101


#### Generate ALL GT images

This cell is for generating all images using all valid GT files in a folder.

In [ ]:
# Generate all onshore GT images

# Base image dimension
len_x, len_y = 1920, 1080

for horizon_key in horizon_gt_files_onshore_dict:
    print(" --- " + horizon_key + " --- ")
    
    # Read GT data
    data = loadmat("../../Dataset/VIS_Onshore/HorizonGT/" + horizon_key + "_HorizonGT")

    # Control de frame number
    frame_number = 0

    # Read line per line of GT file
    # horizon = data frame <frame_number> -> (x, y, cos alpha, sen alpha) - See Explanation of GT files
    for horizon in data['structXML'][0]:
        # Create GT image
        # PIL.Image.new(binary chanel, (x dimension, y dimension))
        gt = PIL.Image.new('1', (len_x, len_y))

        # Create a draw with the image
        draw = ImageDraw.Draw(gt)


        # ------- Create the horizon line -------
        # cosine and sine from GT file horizon line
        c, s = horizon[2], horizon[3]
        # horizon line angle
        rad = math.asin(horizon[3]) - math.radians(90)
        # cosine and sine to plot horizon line
        c, s = math.cos(rad), math.sin(rad)

        # central point
        cx = float(horizon[0][0])
        cy = float(horizon[1][0])

        # start point and end point
        x1 = 0 # start point
        y1 = cy
        x2 = len_x # end point
        y2 = cy

        # rotated points
        xr1 = c*(x1-cx) - s*(y1-cy) + cx
        yr1 = s*(x1-cx) + c*(y1-cy) + cy
        xr2 = c*(x2-cx) - s*(y2-cy) + cx
        yr2 = s*(x2-cx) + c*(y2-cy) + cy
        # ---------------------------------------

        # Draw the horizon line
        # draw.line((x start point, y start point, x end point, y end point), white color, 1 pixel of width)
        draw.line((xr1, yr1, xr2, yr2), fill=1, width=6)

        # Show the image
        #gt.show()

        gt = gt.resize((x_size, y_size))

        # Save the image
        gt.save("GTs/" + horizon_key + "_GT" + str(frame_number) + ".jpg")
        frame_number = frame_number + 1

In [29]:
# Generate all onboard GT images

# Base image dimension
len_x, len_y = 1920, 1080

for horizon_key in horizon_gt_files_onboard_dict:
    print(" --- " + horizon_key + " --- ")
    
    # Read GT data
    data = loadmat("../../Dataset/VIS_Onboard/HorizonGT/" + horizon_key + "_HorizonGT")

    # Control de frame number
    frame_number = 0

    # Read line per line of GT file
    # horizon = data frame <frame_number> -> (x, y, cos alpha, sen alpha) - See Explanation of GT files
    for horizon in data['structXML'][0]:
        # Create GT image
        # PIL.Image.new(binary chanel, (x dimension, y dimension))
        gt = PIL.Image.new('1', (len_x, len_y))

        # Create a draw with the image
        draw = ImageDraw.Draw(gt)


        # ------- Create the horizon line -------
        # cosine and sine from GT file horizon line
        c, s = horizon[2], horizon[3]
        # horizon line angle
        rad = math.asin(horizon[3]) - math.radians(90)
        # cosine and sine to plot horizon line
        c, s = math.cos(rad), math.sin(rad)

        # central point
        cx = float(horizon[0][0])
        cy = float(horizon[1][0])

        # start point and end point
        x1 = 0 # start point
        y1 = cy
        x2 = len_x # end point
        y2 = cy

        # rotated points
        xr1 = c*(x1-cx) - s*(y1-cy) + cx
        yr1 = s*(x1-cx) + c*(y1-cy) + cy
        xr2 = c*(x2-cx) - s*(y2-cy) + cx
        yr2 = s*(x2-cx) + c*(y2-cy) + cy
        # ---------------------------------------

        # Draw the horizon line
        # draw.line((x start point, y start point, x end point, y end point), white color, 1 pixel of width)
        draw.line((xr1, yr1, xr2, yr2), fill=1, width=6)

        # Show the image
        #gt.show()

        gt = gt.resize((x_size, y_size))

        # Save the image
        gt.save("GTs/" + horizon_key + "_GT" + str(frame_number) + ".jpg")
        frame_number = frame_number + 1

 --- MVI_0788_VIS_OB --- 


KeyboardInterrupt: 

In [30]:
-----------------------


Fazer: arrmar ocave ara gerar caracerisicas com mariz

SyntaxError: invalid syntax (<ipython-input-30-261475e7229d>, line 1)